In [16]:
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from googleapiclient.discovery import build
import os
import pandas as pd

load_dotenv()

def pre_processing(ch_id):
    df=pd.DataFrame()  #empty data frame
    api_key=os.getenv('API_KEY')
    yt=build('youtube','v3',developerKey=api_key)

    request=yt.channels().list(
    id=ch_id,
    part=['contentDetails','snippet','statistics']
)
    response=request.execute()

#from snippet
    ch_name=response['items'][0]['snippet']['title']
    ch_desc=response['items'][0]['snippet']['description']
    ch_thumbnail=response['items'][0]['snippet']['thumbnails']['medium']['url']

#from contentDetails
    uploads_id=response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

#from statistics
    subscibers=response['items'][0]['statistics']['subscriberCount']
    total_videos=response['items'][0]['statistics']['videoCount']

#making the embedding LLM object
    model=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)    

#initializing the flag for break condition
    next_page_id=None

#parsing upload playlist videos
    while True:
        request=yt.playlistItems().list(
        playlistId=uploads_id,
        part=['contentDetails'],
        maxResults=20,
        pageToken=next_page_id
    )
        response=request.execute()

        #storing the next page id if any else storing None
        try:next_page_id=response['nextPageToken'] 
        except: next_page_id=None

        #extracting the video ids of a batch of 20 video items
        v_id=[]
        for item in response['items']:
            v_id.append(item['contentDetails']['videoId']) 

        #fetching details of all video ids at once
        request=yt.videos().list(
        id=v_id,
        part=['statistics','contentDetails','snippet']
    )
        response=request.execute()

        #extracting important details of each video id and storing them in respective list
        title=[]
        desc=[]
        thumbnails=[]
        tags=[]
        duration=[]
        views=[]
        likes=[]
        comments=[]
        for item in response['items']:
            try:title.append(item['snippet']['title'])
            except:title.append(None)

            try:desc.append(item['snippet']['description'])
            except:desc.append(None)

            try:thumbnails.append(item['snippet']['thumbnails']['medium']['url'])
            except:thumbnails.append(None)

            try:tags.append(item['snippet']['tags'])
            except:tags.append(None)

            time=item['contentDetails']['duration']
            try:time=f"{time[time.index('T')+1:time.index('H')]}:{time[time.index('H')+1:time.index('M')]}:{time[time.index('M')+1:time.index('S')]}"
            except:
                try:time=f"{time[time.index('T')+1:time.index('M')]}:{time[time.index('M')+1:time.index('S')]}"
                except:
                    try:time=f"00:{time[time.index('T')+1:time.index('S')]}"
                    except:time='00:00'
                
            try:duration.append(time)
            except:duration.append(None)
            
            try:views.append(item['statistics']['viewCount'])
            except:views.append(None)

            try:likes.append(item['statistics']["likeCount"])
            except:likes.append(None)

            try:comments.append(item['statistics']['commentCount'])
            except:comments.append(None)

        vdf=pd.DataFrame(
    {
        'VID':v_id,
        'Title':title,
        'Duration':duration,
        'Views':views,
        'Likes':likes,
        'Comments':comments,
        'Thumbnail':thumbnails,
        'Tags':tags,
        'Description':desc
    }
)
        #converting vdf['Tags'] from list to clean string values
        vdf['Tags']=vdf['Tags'].astype(str).str.translate(str.maketrans('','',"[]'"))

        #adding video URLs
        vdf['URL']=vdf['VID'].apply(lambda x:f"https://www.youtube.com/watch?v={x}")

        #framing a data frame 'temp' to pass on to the embeddings LLM
        temp=vdf['Title']+vdf['Description']+vdf['Tags']

        #passing the content to the embeddings LLM
        embeddings=model.embed_documents(list(temp))
        
        #adding those embeddings to the vdf
        vdf['Embedding']=embeddings

        #end of batch creation

        #now add this batch to the final data frame 'df'
        df=pd.concat([df,vdf])
        print(1)
        print(next_page_id)

        #checking break condition
        if next_page_id==None:
            break
    
    return df

               

In [17]:
a=pre_processing('UCarWcnSwdzAlSTo0RCuhKZg')

1
None


In [18]:
a

VID                                              Title Duration  \
0   HB5WWwvPeW8    DeepSeek AI: The Ultimate Guide for Busy People    24:41   
1   ojXtB3nVe_8  No GPU? No Problem! Running DeepSeek R1 on an ...    25:14   
2   8hKwquqfq5s      Master GenAI in 6 month or less - Here's how!     15:5   
3   y7f_XybBJ-A          Are You Ready for a SIX Figure AI Salary?    00:29   
4   tuEYsRshhHA  Why Generative AI Is Your Fast Track to an AI ...    00:44   
5   lRDiN69FaBw  Machine Learning will kill your career in 2025...     23:1   
6   JDFb4Y9PJnI          Full GenAI RoadMap in 2025 | Job Ready AI    30:40   
7   B-VI3hJbAaI  History of AI  #agi #artificialgeneralintellig...    00:27   
8   VOq9zFFOhiM  How AI is Transforming Our Daily Lives #agi #a...    00:25   
9   dOrvZq_t1RY  The REAL 7 Stages of AI - A Beginner's Guide t...      9:8   
10  ZPGRkFvhOt8  AGI as good as the best HUMAN at Everything #a...    00:57   
11  rav-aIg3MGs  Might and Mind: Strong AI vs. Weak AI #agi #ai...    00:33   
12  57w7AUf9tr8  AGI for Beginners | Can Machines REALLY Think ...     12:7   
13  5C9IvgD8TXY                Start your journey Towards AGI now.     6:20   

    Views Likes Comments                                         Thumbnail  \
0     870    27        9  https://i.ytimg.com/vi/HB5WWwvPeW8/mqdefault.jpg   
1     958    31        6  https://i.ytimg.com/vi/ojXtB3nVe_8/mqdefault.jpg   
2    8255   449       90  https://i.ytimg.com/vi/8hKwquqfq5s/mqdefault.jpg   
3     880    24        3  https://i.ytimg.com/vi/y7f_XybBJ-A/mqdefault.jpg   
4     854    32        2  https://i.ytimg.com/vi/tuEYsRshhHA/mqdefault.jpg   
5   95006  2154      400  https://i.ytimg.com/vi/lRDiN69FaBw/mqdefault.jpg   
6   16079   873      161  https://i.ytimg.com/vi/JDFb4Y9PJnI/mqdefault.jpg   
7     164     7        0  https://i.ytimg.com/vi/B-VI3hJbAaI/mqdefault.jpg   
8     391    12        2  https://i.ytimg.com/vi/VOq9zFFOhiM/mqdefault.jpg   
9     680    21        5  https://i.ytimg.com/vi/dOrvZq_t1RY/mqdefault.jpg   
10    600    18        0  https://i.ytimg.com/vi/ZPGRkFvhOt8/mqdefault.jpg   
11    114     6        0  https://i.ytimg.com/vi/rav-aIg3MGs/mqdefault.jpg   
12   1358    35        5  https://i.ytimg.com/vi/57w7AUf9tr8/mqdefault.jpg   
13    273    17        4  https://i.ytimg.com/vi/5C9IvgD8TXY/mqdefault.jpg   

                                                 Tags  \
0                      deepseek v3, ai, make money ai   
1   ai, artificial intelligence, chatgpt, ai news,...   
2   generative ai tutorial, ai tools, artificial i...   
3                                                None   
4                             ai skills for beginners   
5   "dont waste your time", ai, machine learning, ...   
6   ai, learn generative ai, how to learn generati...   
7   brief history of ai, machine learning, ai hist...   
8   what is ai, generative ai, examples of artific...   
9   artificial intelligence, 7 stages of ai, stage...   
10  ai, machine learning, artificial intelligence,...   
11  what is artificial intelligence, artificial in...   
12  ai, artificial intelligence, singularitynet, a...   
13  AGI, What is AGI vs AI?, Is AGI even possible?...   

                                          Description  \
0   DeepSeek R1, simplified! 🚀 In this ultimate gu...   
1   🚀 Can You Run DeepSeek R1 on a 4-Year-Old Lapt...   
2   Want to master Generative AI in 6 months or le...   
3   🚀 Want a 6-figure AI salary? Master GenAI, bui...   
4   ML will kill your career as a fresher. Learn w...   
5   Machine Learning has been the darling of tech ...   
6   Looking to kickstart a career in Generative AI...   
7   AI has been around way before ChatGPT! The gro...   
8   AI is everywhere in our daily life, from virtu...   
9   🎥 Demystifying AI: A Beginner's Guide to Stage...   
10  AGI will be as good as the best of humans at e...   
11                                                      
12  Ever wonder if robots will become our super-sm...   
1

In [ ]:
request=yt.playlistItems().list(
    playlistId=uploads_id,
    part=['contentDetails'],
    maxResults=50,
)

In [12]:
a.iloc[52]

VID                                                  Do-1zPqmWYw
Title                                 Cutting-Edge Business Idea
Duration                                                   00:40
Views                                                      37377
Likes                                                       1719
Comments                                                      26
Thumbnail       https://i.ytimg.com/vi/Do-1zPqmWYw/mqdefault.jpg
Tags                                                        None
Description                           Cutting-Edge Business Idea
Embedding      [0.02772763930261135, -0.015566593036055565, -...
Name: 12, dtype: object

In [5]:
#from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from googleapiclient.discovery import build
import os
import pandas as pd

#load_dotenv()
api_key=os.getenv('API_KEY')

model=ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite-preview-06-17',google_api_key=api_key)
response=model.invoke('who is kurosaki ichigo,give a 5 sentence answer')
print(response.content)

Kurosaki Ichigo is the protagonist of the popular anime and manga series *Bleach*. He is a human with the ability to see ghosts and later becomes a substitute Soul Reaper, tasked with guiding souls to the afterlife and protecting the living from evil spirits called Hollows. Ichigo possesses immense spiritual power and a strong sense of justice, often putting himself in danger to protect his friends and loved ones. Throughout his journey, he grapples with his own inner demons and the complex world of Soul Reapers, Arrancars, and other supernatural beings. His unwavering determination and growth make him a relatable and compelling hero for fans worldwide.


In [74]:
ch_id='UCarWcnSwdzAlSTo0RCuhKZg'

# This code will be in main.py

In [75]:
yt=build('youtube','v3',developerKey=api_key)

request=yt.channels().list(
id=ch_id,
part=['contentDetails','snippet','statistics']
)
response=request.execute()

#from snippet
ch_name=response['items'][0]['snippet']['title']
ch_desc=response['items'][0]['snippet']['description']
ch_thumbnail=response['items'][0]['snippet']['thumbnails']['medium']['url']

#from contentDetails
uploads_id=response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

#from statistics
subscribers=response['items'][0]['statistics']['subscriberCount']
total_videos=response['items'][0]['statistics']['videoCount']

# This in preprocessor.py

In [279]:
from langchain_google_genai import ChatGoogleGenerativeAI
from googleapiclient.discovery import build
import os
import pandas as pd
import json

api_key=os.getenv('API_KEY')

def pre_processing(ch_id,uploads_id):

#creating empty data frame
    df=pd.DataFrame()  
    
#initializing the flag for break condition
    next_page_id=None

#parsing upload playlist videos
    while True:
        request=yt.playlistItems().list(
        playlistId=uploads_id,
        part=['contentDetails'],
        maxResults=20,
        pageToken=next_page_id
    )
        response=request.execute()

        #storing the next page id if any else storing None
        try:next_page_id=response['nextPageToken'] 
        except: next_page_id=None

        #extracting the video ids of a batch of 20 video items
        v_id=[]
        for item in response['items']:
            v_id.append(item['contentDetails']['videoId']) 

        #fetching details of all video ids at once
        request=yt.videos().list(
        id=v_id,
        part=['statistics','contentDetails','snippet']
    )
        response=request.execute()

        #extracting important details of each video id and storing them in respective list
        title=[]
        desc=[]
        thumbnails=[]
        tags=[]
        duration=[]
        views=[]
        likes=[]
        comments=[]
        date=[]
        for item in response['items']:
            try:title.append(item['snippet']['title'])
            except:title.append(None)

            try:desc.append(item['snippet']['description'])
            except:desc.append(None)

            try:thumbnails.append(item['snippet']['thumbnails']['medium']['url'])
            except:thumbnails.append(None)

            try:tags.append(item['snippet']['tags'])
            except:tags.append(None)

            pub_date=item['snippet']['publishedAt']
            pub_date=pub_date[:pub_date.index('T')]
            try: date.append(pub_date)
            except: date.append(None)

            time=item['contentDetails']['duration']
            try:time=f"{time[time.index('T')+1:time.index('H')]}:{time[time.index('H')+1:time.index('M')]}:{time[time.index('M')+1:time.index('S')]}"
            except:
                try:time=f"{time[time.index('T')+1:time.index('M')]}:{time[time.index('M')+1:time.index('S')]}"
                except:
                    try:time=f"00:{time[time.index('T')+1:time.index('S')]}"
                    except:time='00:00'
                
            try:duration.append(time)
            except:duration.append(None)
            
            try:views.append(item['statistics']['viewCount'])
            except:views.append(None)

            try:likes.append(item['statistics']["likeCount"])
            except:likes.append(None)

            try:comments.append(item['statistics']['commentCount'])
            except:comments.append(None)

        vdf=pd.DataFrame(
    {
        'VID':v_id,
        'Title':title,
        'Duration':duration,
        'Views':views,
        'Likes':likes,
        'Comments':comments,
        'Thumbnail':thumbnails,
        'Tags':tags,
        'Description':desc,
        'Date':date
    }
)
        #converting vdf['Tags'] from list to clean string values
        vdf['Tags']=vdf['Tags'].astype(str).str.translate(str.maketrans('','',"[]'"))

        #adding video URLs
        vdf['URL']=vdf['VID'].apply(lambda x:f"https://www.youtube.com/watch?v={x}")

        #converting the batch to json format before feeding to LLM
        video_data=vdf[['VID','Title','Description','Tags','Views','Likes','Duration','Date']].to_json(orient='records')
        video_data=json.loads(video_data)

        #sending batch to LLM for filtering
        filtered_df=feed_llm(video_data,goals)
        
        #extracting common data in vdf and filtered data and merging on VID
        filtered_df=pd.merge(vdf,filtered_df,on='VID',how='right')
        
        
        #now add this batch to the final data frame 'df'
        df=pd.concat([df,vdf])
        print(1)
        print(next_page_id)

        #checking break condition
        if next_page_id==None:
            break
    
    return df.reset_index(drop=True),filtered_df

               

In [79]:
df,json_format=pre_processing('UCarWcnSwdzAlSTo0RCuhKZg',uploads_id)

1
None


In [256]:
goals='I want to learn the fundamentals of machine learning and understand how real-world ML models are built and deployed'

In [177]:
goals='i want to learn how to dance'

In [261]:
goals="I’m preparing for software engineering internships and want to improve my data structures and algorithms skills."

In [262]:
prompt=f'''You are an intelligent assistant designed to help users find the most relevant YouTube videos based on their learning or personal development goals.

You will receive:
1. A clear description of the user's goals.
2. A list of video metadata, where each video includes:
   - Video ID (VID)
   - Title
   - Description
   - Tags
   - View count
   - Like count
   - Duration
   - Upload date

Your tasks are:
1. Analyze each video’s usefulness with respect to the user’s goals, current trends, and future prospects.
2. Filter out only those videos that are *most relevant and most beneficial* for the user.
3. Categorize each selected video under the appropriate label based on its content.


---

### 🧠 Categories

#### For technical/educational content:
- **Must Watch** – Crucial for the goal; high impact
- **Foundational** – Covers basics or prerequisites
- **Extra Knowledge** – Not essential but interesting
- **Practical Tutorial** – Step-by-step guides or coding walkthroughs
- **Quick Wins** – Easy tricks/tips with instant results
- **Conceptual Clarity** – Explains key ideas clearly
- **Advice** – General guidance from experience
- **Deep Dive** – In-depth exploration of a topic

#### For self-help/personal development:
- **Core Mindset Shift** – Changes perspective significantly
- **Actionable Advice** – Tips that can be directly applied
- **Concept Explainer** – Breaks down personal growth theories
- **Motivational** – Inspires action or energy
- **Habit Building** – Focuses on behavior change
- **Psychological Insight** – Mental models or cognitive insights
- **Extra Perspective** – Broader views that enrich thinking

---

For each selected video, return only a JSON object in the following format:

  "VID": "<video_id>",
  "Category": "<chosen category>",
  "Justification": "<brief explanation of why this video is relevant and categorized as such>"

Only include videos that are genuinely useful for the user's goal.
If **none** of the videos are relevant or valuable, return a single JSON object exactly like this:

  "VID": "NA",
  "category": "NA",
  "justification": "NA"

Do NOT return any other videos in that case.

### USER GOALS:
{goals}

---

### VIDEO DATA:
{json_format}
'''

In [263]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-06-17",google_api_key=api_key,temperature=0.0)
result=model.invoke(prompt)
print(result.content)

```json
[
  {
    "VID": "lRDiN69FaBw",
    "Category": "Advice",
    "Justification": "This video directly addresses the user's goal by advising against traditional ML/Data Science for freshers and suggesting an alternative practical approach for AI jobs, which is highly relevant for internship preparation."
  },
  {
    "VID": "JDFb4Y9PJnI",
    "Category": "Foundational",
    "Justification": "This video provides a comprehensive roadmap for Generative AI, which is a key area in modern AI careers. It helps build a foundational understanding and a path to becoming job-ready, directly supporting internship preparation."
  },
  {
    "VID": "8hKwquqfq5s",
    "Category": "Advice",
    "Justification": "This video offers a strategy to master Generative AI quickly and get started in the AI field as an entry-level candidate, which is excellent advice for someone preparing for internships."
  },
  {
    "VID": "tuEYsRshhHA",
    "Category": "Advice",
    "Justification": "This short video h

In [267]:
text=result.content

In [268]:
print(text)

```json
[
  {
    "VID": "lRDiN69FaBw",
    "Category": "Advice",
    "Justification": "This video directly addresses the user's goal by advising against traditional ML/Data Science for freshers and suggesting an alternative practical approach for AI jobs, which is highly relevant for internship preparation."
  },
  {
    "VID": "JDFb4Y9PJnI",
    "Category": "Foundational",
    "Justification": "This video provides a comprehensive roadmap for Generative AI, which is a key area in modern AI careers. It helps build a foundational understanding and a path to becoming job-ready, directly supporting internship preparation."
  },
  {
    "VID": "8hKwquqfq5s",
    "Category": "Advice",
    "Justification": "This video offers a strategy to master Generative AI quickly and get started in the AI field as an entry-level candidate, which is excellent advice for someone preparing for internships."
  },
  {
    "VID": "tuEYsRshhHA",
    "Category": "Advice",
    "Justification": "This short video h

In [269]:
text=text.replace('json','')
text=text.replace("`",'')

In [270]:
text

'\n[\n  {\n    "VID": "lRDiN69FaBw",\n    "Category": "Advice",\n    "Justification": "This video directly addresses the user\'s goal by advising against traditional ML/Data Science for freshers and suggesting an alternative practical approach for AI jobs, which is highly relevant for internship preparation."\n  },\n  {\n    "VID": "JDFb4Y9PJnI",\n    "Category": "Foundational",\n    "Justification": "This video provides a comprehensive roadmap for Generative AI, which is a key area in modern AI careers. It helps build a foundational understanding and a path to becoming job-ready, directly supporting internship preparation."\n  },\n  {\n    "VID": "8hKwquqfq5s",\n    "Category": "Advice",\n    "Justification": "This video offers a strategy to master Generative AI quickly and get started in the AI field as an entry-level candidate, which is excellent advice for someone preparing for internships."\n  },\n  {\n    "VID": "tuEYsRshhHA",\n    "Category": "Advice",\n    "Justification": "Thi

In [271]:
text=json.loads(text)

In [272]:
text

[{'VID': 'lRDiN69FaBw',
  'Category': 'Advice',
  'Justification': "This video directly addresses the user's goal by advising against traditional ML/Data Science for freshers and suggesting an alternative practical approach for AI jobs, which is highly relevant for internship preparation."},
 {'VID': 'JDFb4Y9PJnI',
  'Category': 'Foundational',
  'Justification': 'This video provides a comprehensive roadmap for Generative AI, which is a key area in modern AI careers. It helps build a foundational understanding and a path to becoming job-ready, directly supporting internship preparation.'},
 {'VID': '8hKwquqfq5s',
  'Category': 'Advice',
  'Justification': 'This video offers a strategy to master Generative AI quickly and get started in the AI field as an entry-level candidate, which is excellent advice for someone preparing for internships.'},
 {'VID': 'tuEYsRshhHA',
  'Category': 'Advice',
  'Justification': 'This short video highlights Generative AI as a fast track to an AI career, pr

In [273]:
d=pd.DataFrame(text)

In [274]:
d

VID            Category  \
0   lRDiN69FaBw              Advice   
1   JDFb4Y9PJnI        Foundational   
2   8hKwquqfq5s              Advice   
3   tuEYsRshhHA              Advice   
4   57w7AUf9tr8  Conceptual Clarity   
5   dOrvZq_t1RY  Conceptual Clarity   
6   B-VI3hJbAaI     Extra Knowledge   
7   VOq9zFFOhiM     Extra Knowledge   
8   ZPGRkFvhOt8     Extra Knowledge   
9   rav-aIg3MGs     Extra Knowledge   
10  5C9IvgD8TXY     Extra Knowledge   
11  y7f_XybBJ-A              Advice   
12  HB5WWwvPeW8     Extra Knowledge   
13  ojXtB3nVe_8     Extra Knowledge   

                                        Justification  
0   This video directly addresses the user's goal ...  
1   This video provides a comprehensive roadmap fo...  
2   This video offers a strategy to master Generat...  
3   This short video highlights Generative AI as a...  
4   This video explains Artificial General Intelli...  
5   This video provides a beginner's guide to the ...  
6   This video offers a brief history of AI, which...  
7   This video explains how AI transforms daily li...  
8   This video defines AGI in a concise manner. Wh...  
9   This video briefly touches on Strong AI vs. We...  
10  This video encourages starting a journey towar...  
11  This short video suggests mastering GenAI and ...  
12  This video is a guide to DeepSeek AI. While it...  
13  This video demonstrates running an AI model lo...

In [275]:
d=pd.merge(df,d,on='VID',how='right')


# second function

In [282]:
def feed_llm(video_data,goals):

    #making LLM object
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite-preview-06-17",google_api_key=api_key,temperature=0.2)  

    #prompt generation
    prompt=prompt=f'''
    You are an intelligent assistant designed to help users find the most relevant YouTube videos based on their learning or personal development goals.
    
    You will receive:
    1. A clear description of the user's goals.
    2. A list of video metadata, where each video includes:
       - Video ID (VID)
       - Title
       - Description
       - Tags
       - View count
       - Like count
       - Duration
       - Upload date
    
    Your tasks are:
    1. Analyze each video’s usefulness with respect to the user’s goals, current trends, and future prospects.
    2. Filter out only the videos that are *most relevant and most beneficial* for the user.
    3. Categorize each selected video under the appropriate label based on its content.
    4. If no videos are useful towards the user's goals return an empty JSON in format:
    
    (
      "VID": 'NA',
      "Category": 'NA',
      "Justification": 'NA'
    )
    ---
    
    ### 🧠 Categories
    
    #### For technical/educational content:
    - **Must Watch** – Crucial for the goal; high impact
    - **Foundational** – Covers basics or prerequisites
    - **Extra Knowledge** – Not essential but interesting
    - **Practical Tutorial** – Step-by-step guides or coding walkthroughs
    - **Quick Wins** – Easy tricks/tips with instant results
    - **Conceptual Clarity** – Explains key ideas clearly
    - **Advice** – General guidance from experience
    - **Deep Dive** – In-depth exploration of a topic
    
    #### For self-help/personal development:
    - **Core Mindset Shift** – Changes perspective significantly
    - **Actionable Advice** – Tips that can be directly applied
    - **Concept Explainer** – Breaks down personal growth theories
    - **Motivational** – Inspires action or energy
    - **Habit Building** – Focuses on behavior change
    - **Psychological Insight** – Mental models or cognitive insights
    - **Extra Perspective** – Broader views that enrich thinking
    
    ---
    
    For each selected video, return a JSON object in the following format:
    (
      "VID": "<video_id>",
      "Category": "<chosen category>",
      "Justification": "<brief explanation of why this video is relevant and categorized as such>"
    )
    
    Only include videos that *genuinely align* with the user's goals and will help them grow or make progress.
    
    ---
    
    ### USER GOALS:
    {goals}
    
    ---
    
    ### VIDEO DATA:
    {json_format}
    '''

    result=model.invoke(prompt)
    filtered_data=result.content
    filtered_data=filtered_data.replace('json','')
    filtered_data=filtered_data.replace('`','')
    filtered_data=json.loads(filtered_data)
    filtered_data=pd.DataFrame(filtered_data)
    return filtered_data

In [243]:
a=pd.DataFrame([{
    'VID':"NA",
    'Extra':"NA"
}])

In [244]:
a

VID Extra
0  NA    NA

In [159]:
a=pd.merge(a,df,on='VID',how='left')

In [171]:
a.isnull()

Empty DataFrame
Columns: [VID, Category, Justification]
Index: []

In [248]:
a

VID category                                      justification
0  NA       NA  None of the provided videos are relevant to le...

In [247]:
a=d[d['VID'].isin(a['VID'])]

In [276]:
d

VID                                              Title Duration  \
0   lRDiN69FaBw  Machine Learning will kill your career in 2025...     23:1   
1   JDFb4Y9PJnI          Full GenAI RoadMap in 2025 | Job Ready AI    30:40   
2   8hKwquqfq5s      Master GenAI in 6 month or less - Here's how!     15:5   
3   tuEYsRshhHA  Why Generative AI Is Your Fast Track to an AI ...    00:44   
4   57w7AUf9tr8  AGI for Beginners | Can Machines REALLY Think ...     12:7   
5   dOrvZq_t1RY  The REAL 7 Stages of AI - A Beginner's Guide t...      9:8   
6   B-VI3hJbAaI  History of AI  #agi #artificialgeneralintellig...    00:27   
7   VOq9zFFOhiM  How AI is Transforming Our Daily Lives #agi #a...    00:25   
8   ZPGRkFvhOt8  AGI as good as the best HUMAN at Everything #a...    00:57   
9   rav-aIg3MGs  Might and Mind: Strong AI vs. Weak AI #agi #ai...    00:33   
10  5C9IvgD8TXY                Start your journey Towards AGI now.     6:20   
11  y7f_XybBJ-A          Are You Ready for a SIX Figure AI Salary?    00:29   
12  HB5WWwvPeW8    DeepSeek AI: The Ultimate Guide for Busy People    24:41   
13  ojXtB3nVe_8  No GPU? No Problem! Running DeepSeek R1 on an ...    25:14   

    Views Likes Comments                                         Thumbnail  \
0   95084  2156      401  https://i.ytimg.com/vi/lRDiN69FaBw/mqdefault.jpg   
1   16085   873      161  https://i.ytimg.com/vi/JDFb4Y9PJnI/mqdefault.jpg   
2    8258   449       90  https://i.ytimg.com/vi/8hKwquqfq5s/mqdefault.jpg   
3     854    32        2  https://i.ytimg.com/vi/tuEYsRshhHA/mqdefault.jpg   
4    1358    35        5  https://i.ytimg.com/vi/57w7AUf9tr8/mqdefault.jpg   
5     681    21        5  https://i.ytimg.com/vi/dOrvZq_t1RY/mqdefault.jpg   
6     164     7        0  https://i.ytimg.com/vi/B-VI3hJbAaI/mqdefault.jpg   
7     391    12        2  https://i.ytimg.com/vi/VOq9zFFOhiM/mqdefault.jpg   
8     600    18        0  https://i.ytimg.com/vi/ZPGRkFvhOt8/mqdefault.jpg   
9     114     6        0  https://i.ytimg.com/vi/rav-aIg3MGs/mqdefault.jpg   
10    273    17        4  https://i.ytimg.com/vi/5C9IvgD8TXY/mqdefault.jpg   
11    881    24        3  https://i.ytimg.com/vi/y7f_XybBJ-A/mqdefault.jpg   
12    871    27        9  https://i.ytimg.com/vi/HB5WWwvPeW8/mqdefault.jpg   
13    959    31        6  https://i.ytimg.com/vi/ojXtB3nVe_8/mqdefault.jpg   

                                                 Tags  \
0   "dont waste your time", ai, machine learning, ...   
1   ai, learn generative ai, how to learn generati...   
2   generative ai tutorial, ai tools, artificial i...   
3                             ai skills for beginners   
4   ai, artificial intelligence, singularitynet, a...   
5   artificial intelligence, 7 stages of ai, stage...   
6   brief history of ai, machine learning, ai hist...   
7   what is ai, generative ai, examples of artific...   
8   ai, machine learning, artificial intelligence,...   
9   what is artificial intelligence, artificial in...   
10  AGI, What is AGI vs AI?, Is AGI even possible?...   
11                                               None   
12                     deepseek v3, ai, make money ai   
13  ai, artificial intelligence, chatgpt, ai news,...   

                                          Description        Date  \
0   Machine Learning has been the darling of tech ...  2025-01-05   
1   Looking to kickstart a career in Generative AI...  2024-09-20   
2   Want to master Generative AI in 6 months or le...  2025-01-23   
3   ML will kill your career as a fresher. Learn w...  2025-01-10   
4   Ever wonder if robots will become our super-sm...  2024-05-08   
5   🎥 Demystifying AI: A Beginner's Guide to Stage...  2024-06-16   
6   AI has been around way before ChatGPT! The gro...  2024-06-18   
7   AI is everywhere in our daily life, from virtu...  2024-06-17   
8   AGI will be as good as the best of humans at e...  2024-05-15   
9                                                      2024-05-09   
10  In this ocean of AI channels, I inte

In [283]:
df,d=pre_processing(ch_id,uploads_id)

1
None


In [284]:
df

VID                                              Title Duration  \
0   HB5WWwvPeW8    DeepSeek AI: The Ultimate Guide for Busy People    24:41   
1   ojXtB3nVe_8  No GPU? No Problem! Running DeepSeek R1 on an ...    25:14   
2   8hKwquqfq5s      Master GenAI in 6 month or less - Here's how!     15:5   
3   y7f_XybBJ-A          Are You Ready for a SIX Figure AI Salary?    00:29   
4   tuEYsRshhHA  Why Generative AI Is Your Fast Track to an AI ...    00:44   
5   lRDiN69FaBw  Machine Learning will kill your career in 2025...     23:1   
6   JDFb4Y9PJnI          Full GenAI RoadMap in 2025 | Job Ready AI    30:40   
7   B-VI3hJbAaI  History of AI  #agi #artificialgeneralintellig...    00:27   
8   VOq9zFFOhiM  How AI is Transforming Our Daily Lives #agi #a...    00:25   
9   dOrvZq_t1RY  The REAL 7 Stages of AI - A Beginner's Guide t...      9:8   
10  ZPGRkFvhOt8  AGI as good as the best HUMAN at Everything #a...    00:57   
11  rav-aIg3MGs  Might and Mind: Strong AI vs. Weak AI #agi #ai...    00:33   
12  57w7AUf9tr8  AGI for Beginners | Can Machines REALLY Think ...     12:7   
13  5C9IvgD8TXY                Start your journey Towards AGI now.     6:20   

    Views Likes Comments                                         Thumbnail  \
0     871    27        9  https://i.ytimg.com/vi/HB5WWwvPeW8/mqdefault.jpg   
1     959    31        6  https://i.ytimg.com/vi/ojXtB3nVe_8/mqdefault.jpg   
2    8261   449       90  https://i.ytimg.com/vi/8hKwquqfq5s/mqdefault.jpg   
3     881    24        3  https://i.ytimg.com/vi/y7f_XybBJ-A/mqdefault.jpg   
4     854    32        2  https://i.ytimg.com/vi/tuEYsRshhHA/mqdefault.jpg   
5   95088  2156      400  https://i.ytimg.com/vi/lRDiN69FaBw/mqdefault.jpg   
6   16087   873      161  https://i.ytimg.com/vi/JDFb4Y9PJnI/mqdefault.jpg   
7     164     7        0  https://i.ytimg.com/vi/B-VI3hJbAaI/mqdefault.jpg   
8     391    12        2  https://i.ytimg.com/vi/VOq9zFFOhiM/mqdefault.jpg   
9     681    21        5  https://i.ytimg.com/vi/dOrvZq_t1RY/mqdefault.jpg   
10    600    18        0  https://i.ytimg.com/vi/ZPGRkFvhOt8/mqdefault.jpg   
11    114     6        0  https://i.ytimg.com/vi/rav-aIg3MGs/mqdefault.jpg   
12   1358    35        5  https://i.ytimg.com/vi/57w7AUf9tr8/mqdefault.jpg   
13    273    17        4  https://i.ytimg.com/vi/5C9IvgD8TXY/mqdefault.jpg   

                                                 Tags  \
0                      deepseek v3, ai, make money ai   
1   ai, artificial intelligence, chatgpt, ai news,...   
2   generative ai tutorial, ai tools, artificial i...   
3                                                None   
4                             ai skills for beginners   
5   "dont waste your time", ai, machine learning, ...   
6   ai, learn generative ai, how to learn generati...   
7   brief history of ai, machine learning, ai hist...   
8   what is ai, generative ai, examples of artific...   
9   artificial intelligence, 7 stages of ai, stage...   
10  ai, machine learning, artificial intelligence,...   
11  what is artificial intelligence, artificial in...   
12  ai, artificial intelligence, singularitynet, a...   
13  AGI, What is AGI vs AI?, Is AGI even possible?...   

                                          Description        Date  \
0   DeepSeek R1, simplified! 🚀 In this ultimate gu...  2025-02-28   
1   🚀 Can You Run DeepSeek R1 on a 4-Year-Old Lapt...  2025-02-15   
2   Want to master Generative AI in 6 months or le...  2025-01-23   
3   🚀 Want a 6-figure AI salary? Master GenAI, bui...  2025-01-15   
4   ML will kill your career as a fresher. Learn w...  2025-01-10   
5   Machine Learning has been the darling of tech ...  2025-01-05   
6   Looking to kickstart a career in Generative AI...  2024-09-20   
7   AI has been around way before ChatGPT! The gro...  2024-06-18   
8   AI is everywhere in our daily life, from virtu...  2024-06-17   
9   🎥 Demystifying AI: A Beginner's Guide to Stage...  2024-06-16   
10  AGI will be as good as the best of h

In [285]:
d

VID                                              Title Duration  \
0  lRDiN69FaBw  Machine Learning will kill your career in 2025...     23:1   
1  JDFb4Y9PJnI          Full GenAI RoadMap in 2025 | Job Ready AI    30:40   
2  8hKwquqfq5s      Master GenAI in 6 month or less - Here's how!     15:5   
3  tuEYsRshhHA  Why Generative AI Is Your Fast Track to an AI ...    00:44   
4  y7f_XybBJ-A          Are You Ready for a SIX Figure AI Salary?    00:29   

   Views Likes Comments                                         Thumbnail  \
0  95088  2156      400  https://i.ytimg.com/vi/lRDiN69FaBw/mqdefault.jpg   
1  16087   873      161  https://i.ytimg.com/vi/JDFb4Y9PJnI/mqdefault.jpg   
2   8261   449       90  https://i.ytimg.com/vi/8hKwquqfq5s/mqdefault.jpg   
3    854    32        2  https://i.ytimg.com/vi/tuEYsRshhHA/mqdefault.jpg   
4    881    24        3  https://i.ytimg.com/vi/y7f_XybBJ-A/mqdefault.jpg   

                                                Tags  \
0  "dont waste your time", ai, machine learning, ...   
1  ai, learn generative ai, how to learn generati...   
2  generative ai tutorial, ai tools, artificial i...   
3                            ai skills for beginners   
4                                               None   

                                         Description        Date  \
0  Machine Learning has been the darling of tech ...  2025-01-05   
1  Looking to kickstart a career in Generative AI...  2024-09-20   
2  Want to master Generative AI in 6 months or le...  2025-01-23   
3  ML will kill your career as a fresher. Learn w...  2025-01-10   
4  🚀 Want a 6-figure AI salary? Master GenAI, bui...  2025-01-15   

                                           URL            Category  \
0  https://www.youtube.com/watch?v=lRDiN69FaBw              Advice   
1  https://www.youtube.com/watch?v=JDFb4Y9PJnI        Foundational   
2  https://www.youtube.com/watch?v=8hKwquqfq5s   Actionable Advice   
3  https://www.youtube.com/watch?v=tuEYsRshhHA  Conceptual Clarity   
4  https://www.youtube.com/watch?v=y7f_XybBJ-A        Motivational   

                                       Justification  
0  This video directly addresses the user's goal ...  
1  This video offers a comprehensive roadmap for ...  
2  This video provides a framework for mastering ...  
3  This video explains why Generative AI is a fas...  
4  While short, this video aims to motivate users...